# Explanations

- This notebook focuses on pyod models

- The previous notebook showed that anomaly detection models work quite well.

- Looking in the pyod document, I found the benchmarks with this paper:
  - Benchmark: https://pyod.readthedocs.io/en/latest/benchmark.html
  - Paper: https://arxiv.org/abs/2206.09426

- Here are some conclusions from the paper:
  - The performance of unsupervised algorithms depends on the data (no absolute winners).
  - semi-supervised methods can outperform unsupervised methods with a small amount of labelled data (common)
  - unsupervised may outperform supervised under certain conditions (not common)
  - Models that I will be using (Best performing models from the paper):
    - Unsupervised: IForest, COPOD, KNN, CBLOF
    - Supervised: CatB, LGBM

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
import plotly.graph_objects as go
from pyod.models.iforest import IForest
from pyod.models.copod import COPOD
from pyod.models.knn import KNN
from pyod.models.cblof import CBLOF
from pyod.models.ocsvm import OCSVM
import optuna


import scripts.feature as feat
import scripts.utility as util


In [2]:
class FirmEncoder:
    def __init__(self, firms):
        self.encoder = LabelEncoder()
        self.encoder.fit(firms)

        self.scaler = StandardScaler()
        self.scaler.fit(self.encoder.transform(firms).reshape(-1, 1))
    
    def encode(self, firm):
        code = self.encoder.transform([firm]).reshape(-1, 1)
        return self.scaler.transform(code)[0][0]

In [3]:
def multi_ticker_dataset_pipeline(ticker_list, obj=[5, 0.05], normalize=False, extend=None):
    encoder = FirmEncoder(ticker_list)
    X_train, y_train, X_val, y_val = [], [], [], []

    for ticker in ticker_list:
        df = yf.download(ticker, period="5y", interval="1d", progress=False)
        df = df.drop(columns=["Adj Close"])
        firm = encoder.encode(ticker)

        Xtrain, ytrain, Xval, yval = feat.dataset_pipeline(df, 
                                                           obj=obj, 
                                                           norm=normalize, 
                                                           seq_len=None, 
                                                           extend=extend, 
                                                           firm=firm)
            
        X_train.append(Xtrain)
        y_train.append(ytrain)
        X_val.append(Xval)
        y_val.append(yval)

    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)
    X_val = np.vstack(X_val)
    y_val = np.hstack(y_val)

    return X_train, y_train, X_val, y_val

def data_prep_for_tuning(ticker_list, normalize, obj=[5, 0.05]):
    X_train, y_train, X_val, y_val = multi_ticker_dataset_pipeline(ticker_list, obj=obj, normalize=normalize, extend=3)
    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

    y_train = np.where(y_train <= 1, 0, 1)
    y_val = np.where(y_val <= 1, 0, 1)

    print(np.unique(y_train, return_counts=True))
    print(np.unique(y_val, return_counts=True))

    return X_train, y_train, X_val, y_val

In [4]:
def single_class_report(y_true, y_pred, label):
    instance = np.where(y_true == label)[0]
    predict = np.where(y_pred == label)[0]
    correct = np.intersect1d(instance, predict) 

    precision = len(correct) / (len(predict) + 0.01)
    recall = len(correct) / (len(instance)+ 0.01)
    
    def fbeta_score(b):
        return (1+b**2) * (precision * recall) / (b**2 * precision + recall + 0.01)
    f1 = fbeta_score(b=1)
    fbeta = fbeta_score(b=0.5)  # 0.5 beta focus more on precision

    # print(f"----- For Class {label} -----")
    # print(f"Recall: {recall}")
    # print(f"Precision: {precision}")
    # print(f"f1_score: {f1}")
    # print(f"f0.5_score: {fbeta}")

    return [recall, precision, f1, fbeta]


def quick_model_evaluation(model, X_train, y_train, X_val, y_val, label):
    model.fit(X_train, y_train)

    print("train set")
    y_pred = model.predict(X_train)
    train_result = single_class_report(y_train, y_pred, label=label)

    print("val set")
    y_pred = model.predict(X_val)
    val_result = single_class_report(y_val, y_pred, label=label)

    return np.round(train_result + val_result, 3)


def quick_unsupervised_evaluation(model, X_train, y_train, X_val, y_val, label):
    model.fit(X_train)

    # class 1 is normal, -1 is anomaly for sklearn
    # class 0 is normal,  1 is anomaly for pyod
    # print("train set")
    y_pred = model.predict(X_train)
    # y_pred = np.where(y_pred > 0, 0, 1)
    train_result = single_class_report(y_train, y_pred, label=label)

    # print("val set")
    y_pred = model.predict(X_val)
    # y_pred = np.where(y_pred > 0, 0, 1)
    val_result = single_class_report(y_val, y_pred, label=label)

    return np.round(train_result + val_result, 3)

def update_result_table(df, result, idx_name):
    new = pd.DataFrame(data=[result],
                       columns=["train_recall", "train_precision", "train_f1", "train_fbeta",
                                "val_recall", "val_precision", "val_f1", "val_fbeta"],
                       index=[idx_name])
    df = pd.concat([df, new])
    return df


# optuna tuning


In [5]:
def objective_iforest(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 300)
    max_samples = trial.suggest_categorical("max_samples", [0.6, 0.8, 1.0])
    contamination = trial.suggest_float("contamination", 0.1, 0.5)

    model = IForest(n_estimators=n_estimators, max_samples=max_samples, contamination=contamination)
    model.fit(X_train)

    y_pred = model.predict(X_val)
    score = single_class_report(y_val, y_pred, label=1)

    return score[3]

def objective_copod(trial):
    contamination = trial.suggest_float("contamination", 0.1, 0.5)
    
    model = COPOD(contamination=contamination)
    model.fit(X_train)
    
    y_pred = model.predict(X_val)
    score = single_class_report(y_val, y_pred, label=1)
    
    return score[3]

def objective_knn(trial):
    n_neighbors = trial.suggest_int("n_neighbors", 5, 20)
    contamination = trial.suggest_float("contamination", 0.1, 0.5)

    model = KNN(n_neighbors=n_neighbors, contamination=contamination)
    model.fit(X_train)
    
    y_pred = model.predict(X_val)
    score = single_class_report(y_val, y_pred, label=1)
    
    return score[3]

def objective_cblof(trial):
    n_clusters = trial.suggest_int("n_clusters", 7, 10)
    contamination = trial.suggest_float("contamination", 0.1, 0.5)
    alpha = trial.suggest_float("alpha", 0.5, 0.9)
    beta = trial.suggest_int("beta", 3, 7)

    model = CBLOF(n_clusters=n_clusters, contamination=contamination, alpha=alpha, beta=beta)
    model.fit(X_train)
    
    y_pred = model.predict(X_val)
    score = single_class_report(y_val, y_pred, label=1)
    
    return score[3]

def objective_ocsvm(trial):
    kernel = trial.suggest_categorical("kernel", ["poly", "rbf", "sigmoid"])
    gamma = trial.suggest_float("gamma", 1e-5, 1e2, log=True)
    nu = trial.suggest_float("nu", 0.1, 0.5)

    model = OCSVM(kernel=kernel, gamma=gamma, nu=nu)
    model.fit(X_train)
    
    y_pred = model.predict(X_val)
    score = single_class_report(y_val, y_pred, label=1)
    
    return score[3]

def tune_all_models(n_trials):
    models = {"iforest": objective_iforest,
              "copod": objective_copod,
              "knn": objective_knn,
              "cblof": objective_cblof,
              "ocsvm": objective_ocsvm}
    
    studies = {}

    for name, model in models.items():
        study = optuna.create_study(direction="maximize")
        study.optimize(model, n_trials=n_trials)
        studies[name] = study
    
    return studies

# Tickers (Settings)

In [6]:
ticker_list = ["TSLA", "NVDA", "AMZN", "AAPL", "MSFT", "AMD", "GOOG", "META", "NFLX"]
# ticker_list = ["TSLA", "NVDA", "AMZN"]

n_trials = 30

# Binary classification w norm data

## goal: 5 day 5% gain (high anomaly %)

In [7]:
X_train, y_train, X_val, y_val = data_prep_for_tuning(ticker_list, normalize=True, obj=[5, 0.05])

(8802, 101) (8802,) (2205, 101) (2205,)
(array([0, 1]), array([5904, 2898], dtype=int64))
(array([0, 1]), array([1640,  565], dtype=int64))


In [8]:
studies = tune_all_models(n_trials)

for name, study in studies.items():
    print(name)
    print(study.best_trial)

[I 2024-07-23 17:20:08,928] A new study created in memory with name: no-name-86b53ccd-ffb7-4835-8ca7-a9d6f5955821
[I 2024-07-23 17:20:09,650] Trial 0 finished with value: 0.3160909387166379 and parameters: {'n_estimators': 124, 'max_samples': 0.8, 'contamination': 0.4609374387983899}. Best is trial 0 with value: 0.3160909387166379.
[I 2024-07-23 17:20:10,893] Trial 1 finished with value: 0.2617851695049879 and parameters: {'n_estimators': 244, 'max_samples': 0.8, 'contamination': 0.10991794630566908}. Best is trial 0 with value: 0.3160909387166379.
[I 2024-07-23 17:20:11,477] Trial 2 finished with value: 0.30995765638390826 and parameters: {'n_estimators': 124, 'max_samples': 0.6, 'contamination': 0.2609373067504712}. Best is trial 0 with value: 0.3160909387166379.
[I 2024-07-23 17:20:12,455] Trial 3 finished with value: 0.2819069506195617 and parameters: {'n_estimators': 205, 'max_samples': 0.8, 'contamination': 0.15332087964171795}. Best is trial 0 with value: 0.3160909387166379.
[I 

iforest
FrozenTrial(number=28, state=TrialState.COMPLETE, values=[0.3388962836063448], datetime_start=datetime.datetime(2024, 7, 23, 17, 20, 40, 561573), datetime_complete=datetime.datetime(2024, 7, 23, 17, 20, 41, 827152), params={'n_estimators': 275, 'max_samples': 0.6, 'contamination': 0.475870321652919}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=100, step=1), 'max_samples': CategoricalDistribution(choices=(0.6, 0.8, 1.0)), 'contamination': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, trial_id=28, value=None)
copod
FrozenTrial(number=18, state=TrialState.COMPLETE, values=[0.3438733443600308], datetime_start=datetime.datetime(2024, 7, 23, 17, 20, 53, 226844), datetime_complete=datetime.datetime(2024, 7, 23, 17, 20, 53, 755968), params={'contamination': 0.4329351255405163}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'contamination': FloatDistribution(high=0

## goal: 3 day 5% gain (mid anomaly %)

In [9]:
X_train, y_train, X_val, y_val = data_prep_for_tuning(ticker_list, normalize=True, obj=[3, 0.05])

(8802, 101) (8802,) (2205, 101) (2205,)
(array([0, 1]), array([6790, 2012], dtype=int64))
(array([0, 1]), array([1862,  343], dtype=int64))


In [10]:
studies = tune_all_models(n_trials)

for name, study in studies.items():
    print(name)
    print(study.best_trial)

[I 2024-07-23 17:28:57,993] A new study created in memory with name: no-name-1c73b7ff-bb20-4aeb-a773-d0ba84d0f30c
[I 2024-07-23 17:28:59,127] Trial 0 finished with value: 0.24823088563012619 and parameters: {'n_estimators': 236, 'max_samples': 0.8, 'contamination': 0.2348255761974805}. Best is trial 0 with value: 0.24823088563012619.
[I 2024-07-23 17:29:00,324] Trial 1 finished with value: 0.2511115935617937 and parameters: {'n_estimators': 222, 'max_samples': 0.6, 'contamination': 0.19478063718253402}. Best is trial 1 with value: 0.2511115935617937.
[I 2024-07-23 17:29:01,073] Trial 2 finished with value: 0.22792092847328116 and parameters: {'n_estimators': 161, 'max_samples': 0.6, 'contamination': 0.12394579201494303}. Best is trial 1 with value: 0.2511115935617937.
[I 2024-07-23 17:29:02,362] Trial 3 finished with value: 0.23907872186692755 and parameters: {'n_estimators': 279, 'max_samples': 0.6, 'contamination': 0.2835183554479974}. Best is trial 1 with value: 0.2511115935617937.


iforest
FrozenTrial(number=21, state=TrialState.COMPLETE, values=[0.2555248074978749], datetime_start=datetime.datetime(2024, 7, 23, 17, 29, 20, 91564), datetime_complete=datetime.datetime(2024, 7, 23, 17, 29, 21, 243461), params={'n_estimators': 234, 'max_samples': 0.8, 'contamination': 0.1938540778191192}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=100, step=1), 'max_samples': CategoricalDistribution(choices=(0.6, 0.8, 1.0)), 'contamination': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, trial_id=21, value=None)
copod
FrozenTrial(number=28, state=TrialState.COMPLETE, values=[0.23702770608340554], datetime_start=datetime.datetime(2024, 7, 23, 17, 29, 44, 918234), datetime_complete=datetime.datetime(2024, 7, 23, 17, 29, 45, 443831), params={'contamination': 0.37573338207717444}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'contamination': FloatDistribution(high

## goal: 3 day 7% gain (low anomaly %)

In [11]:
X_train, y_train, X_val, y_val = data_prep_for_tuning(ticker_list, normalize=True, obj=[3, 0.07])

(8802, 101) (8802,) (2205, 101) (2205,)
(array([0, 1]), array([7698, 1104], dtype=int64))
(array([0, 1]), array([2027,  178], dtype=int64))


In [12]:
studies = tune_all_models(n_trials)

for name, study in studies.items():
    print(name)
    print(study.best_trial)

[I 2024-07-23 17:34:22,321] A new study created in memory with name: no-name-59b2c4d4-9781-42c6-9d40-55bb1dc8dd4f
[I 2024-07-23 17:34:23,349] Trial 0 finished with value: 0.14174967734128444 and parameters: {'n_estimators': 207, 'max_samples': 0.8, 'contamination': 0.2577802934679778}. Best is trial 0 with value: 0.14174967734128444.
[I 2024-07-23 17:34:24,429] Trial 1 finished with value: 0.12262336480777028 and parameters: {'n_estimators': 238, 'max_samples': 0.6, 'contamination': 0.46077487155981967}. Best is trial 0 with value: 0.14174967734128444.
[I 2024-07-23 17:34:25,297] Trial 2 finished with value: 0.12484472719834493 and parameters: {'n_estimators': 190, 'max_samples': 0.6, 'contamination': 0.4315223470927103}. Best is trial 0 with value: 0.14174967734128444.
[I 2024-07-23 17:34:26,514] Trial 3 finished with value: 0.12632626861219878 and parameters: {'n_estimators': 266, 'max_samples': 0.6, 'contamination': 0.49964585841838494}. Best is trial 0 with value: 0.141749677341284

iforest
FrozenTrial(number=21, state=TrialState.COMPLETE, values=[0.15740547886911066], datetime_start=datetime.datetime(2024, 7, 23, 17, 34, 40, 600561), datetime_complete=datetime.datetime(2024, 7, 23, 17, 34, 41, 488897), params={'n_estimators': 181, 'max_samples': 0.8, 'contamination': 0.1598373213173198}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=100, step=1), 'max_samples': CategoricalDistribution(choices=(0.6, 0.8, 1.0)), 'contamination': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, trial_id=21, value=None)
copod
FrozenTrial(number=19, state=TrialState.COMPLETE, values=[0.15893570940295043], datetime_start=datetime.datetime(2024, 7, 23, 17, 34, 59, 209230), datetime_complete=datetime.datetime(2024, 7, 23, 17, 34, 59, 730640), params={'contamination': 0.18132522270078053}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'contamination': FloatDistribution(hi

# Binary classification w raw data

## goal: 5 day 5% gain (high anomaly %)

In [13]:
X_train, y_train, X_val, y_val = data_prep_for_tuning(ticker_list, normalize=False, obj=[5, 0.05])

(8802, 101) (8802,) (2205, 101) (2205,)
(array([0, 1]), array([5904, 2898], dtype=int64))
(array([0, 1]), array([1640,  565], dtype=int64))


### Best performance

In [14]:
studies = tune_all_models(n_trials)

for name, study in studies.items():
    print(name)
    print(study.best_trial)

[I 2024-07-23 17:39:50,673] A new study created in memory with name: no-name-00a7c660-2f73-463a-9bca-5de544918425
[I 2024-07-23 17:39:51,774] Trial 0 finished with value: 0.38463391277969416 and parameters: {'n_estimators': 229, 'max_samples': 1.0, 'contamination': 0.42677925892849555}. Best is trial 0 with value: 0.38463391277969416.
[I 2024-07-23 17:39:52,246] Trial 1 finished with value: 0.3993308755934558 and parameters: {'n_estimators': 113, 'max_samples': 0.6, 'contamination': 0.377937805797361}. Best is trial 1 with value: 0.3993308755934558.
[I 2024-07-23 17:39:53,059] Trial 2 finished with value: 0.3827350091024517 and parameters: {'n_estimators': 185, 'max_samples': 0.8, 'contamination': 0.4766244094637965}. Best is trial 1 with value: 0.3993308755934558.
[I 2024-07-23 17:39:53,809] Trial 3 finished with value: 0.3256846255585845 and parameters: {'n_estimators': 168, 'max_samples': 0.8, 'contamination': 0.22168550193510483}. Best is trial 1 with value: 0.3993308755934558.
[I 

iforest
FrozenTrial(number=18, state=TrialState.COMPLETE, values=[0.417408051971138], datetime_start=datetime.datetime(2024, 7, 23, 17, 40, 7, 400804), datetime_complete=datetime.datetime(2024, 7, 23, 17, 40, 8, 125191), params={'n_estimators': 167, 'max_samples': 0.6, 'contamination': 0.34103922173525}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=100, step=1), 'max_samples': CategoricalDistribution(choices=(0.6, 0.8, 1.0)), 'contamination': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, trial_id=18, value=None)
copod
FrozenTrial(number=28, state=TrialState.COMPLETE, values=[0.3831753307178901], datetime_start=datetime.datetime(2024, 7, 23, 17, 40, 30, 918881), datetime_complete=datetime.datetime(2024, 7, 23, 17, 40, 31, 430623), params={'contamination': 0.4704577839487078}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'contamination': FloatDistribution(high=0.5, 

## goal: 3 day 5% gain (mid anomaly %)

In [15]:
X_train, y_train, X_val, y_val = data_prep_for_tuning(ticker_list, normalize=False, obj=[3, 0.05])

(8802, 101) (8802,) (2205, 101) (2205,)
(array([0, 1]), array([6790, 2012], dtype=int64))
(array([0, 1]), array([1862,  343], dtype=int64))


In [16]:
studies = tune_all_models(n_trials)

for name, study in studies.items():
    print(name)
    print(study.best_trial)

[I 2024-07-23 17:53:47,662] A new study created in memory with name: no-name-fcf49d66-42fc-4d71-ab34-e5bf89a99d6e
[I 2024-07-23 17:53:48,350] Trial 0 finished with value: 0.2801254667089825 and parameters: {'n_estimators': 153, 'max_samples': 0.8, 'contamination': 0.4571865114343937}. Best is trial 0 with value: 0.2801254667089825.
[I 2024-07-23 17:53:49,276] Trial 1 finished with value: 0.22481017216546012 and parameters: {'n_estimators': 204, 'max_samples': 0.8, 'contamination': 0.13078498588523027}. Best is trial 0 with value: 0.2801254667089825.
[I 2024-07-23 17:53:50,267] Trial 2 finished with value: 0.30620637723646016 and parameters: {'n_estimators': 234, 'max_samples': 0.6, 'contamination': 0.29149128258607715}. Best is trial 2 with value: 0.30620637723646016.
[I 2024-07-23 17:53:51,145] Trial 3 finished with value: 0.2983465609246818 and parameters: {'n_estimators': 205, 'max_samples': 0.6, 'contamination': 0.2997972894640323}. Best is trial 2 with value: 0.30620637723646016.


iforest
FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.30620637723646016], datetime_start=datetime.datetime(2024, 7, 23, 17, 53, 49, 277433), datetime_complete=datetime.datetime(2024, 7, 23, 17, 53, 50, 266665), params={'n_estimators': 234, 'max_samples': 0.6, 'contamination': 0.29149128258607715}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=100, step=1), 'max_samples': CategoricalDistribution(choices=(0.6, 0.8, 1.0)), 'contamination': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, trial_id=2, value=None)
copod
FrozenTrial(number=24, state=TrialState.COMPLETE, values=[0.28909463062361523], datetime_start=datetime.datetime(2024, 7, 23, 17, 54, 27, 301861), datetime_complete=datetime.datetime(2024, 7, 23, 17, 54, 27, 816528), params={'contamination': 0.18126134533386837}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'contamination': FloatDistribution(hig

## goal: 3 day 7% gain (low anomaly %)

In [17]:
X_train, y_train, X_val, y_val = data_prep_for_tuning(ticker_list, normalize=False, obj=[3, 0.07])

(8802, 101) (8802,) (2205, 101) (2205,)
(array([0, 1]), array([7698, 1104], dtype=int64))
(array([0, 1]), array([2027,  178], dtype=int64))


In [18]:
studies = tune_all_models(n_trials)

for name, study in studies.items():
    print(name)
    print(study.best_trial)

[I 2024-07-23 18:05:07,559] A new study created in memory with name: no-name-c74e0a5b-e78e-4742-87a6-ce07ef4b58d3
[I 2024-07-23 18:05:08,475] Trial 0 finished with value: 0.15027018780126916 and parameters: {'n_estimators': 210, 'max_samples': 0.6, 'contamination': 0.11498342807455475}. Best is trial 0 with value: 0.15027018780126916.
[I 2024-07-23 18:05:09,443] Trial 1 finished with value: 0.18842519366104282 and parameters: {'n_estimators': 217, 'max_samples': 0.8, 'contamination': 0.2962144156018103}. Best is trial 1 with value: 0.18842519366104282.
[I 2024-07-23 18:05:10,339] Trial 2 finished with value: 0.18384518309959333 and parameters: {'n_estimators': 201, 'max_samples': 0.8, 'contamination': 0.17577934357599806}. Best is trial 1 with value: 0.18842519366104282.
[I 2024-07-23 18:05:10,853] Trial 3 finished with value: 0.16142737600780052 and parameters: {'n_estimators': 119, 'max_samples': 0.6, 'contamination': 0.40995138428596245}. Best is trial 1 with value: 0.18842519366104

iforest
FrozenTrial(number=24, state=TrialState.COMPLETE, values=[0.20404425473634047], datetime_start=datetime.datetime(2024, 7, 23, 18, 5, 27, 19151), datetime_complete=datetime.datetime(2024, 7, 23, 18, 5, 27, 667365), params={'n_estimators': 149, 'max_samples': 0.6, 'contamination': 0.2570517204795274}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=100, step=1), 'max_samples': CategoricalDistribution(choices=(0.6, 0.8, 1.0)), 'contamination': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, trial_id=24, value=None)
copod
FrozenTrial(number=27, state=TrialState.COMPLETE, values=[0.2080997031995401], datetime_start=datetime.datetime(2024, 7, 23, 18, 5, 45, 115591), datetime_complete=datetime.datetime(2024, 7, 23, 18, 5, 45, 630336), params={'contamination': 0.17842898211901728}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'contamination': FloatDistribution(high=0.5